In [ ]:
import cv2
import numpy as np
import glob

# 5 x 6 '칸' → 내부 코너는 4 x 5
CHECKERBOARD = (4, 5)  # (가로 코너 수, 세로 코너 수)

# 3D object points (체커보드를 z=0 평면에 놓았다고 가정)
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

objpoints = []  # 3D 점들
imgpoints = []  # 2D 이미지 점들

images = sorted(glob.glob("capture/*.jpg"))

print(f"Found {len(images)} images")

for fname in images:
    img = cv2.imread(fname)
    if img is None:
        print(f"Cannot read {fname}")
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 체커보드 코너 찾기 (4 x 5 코너)
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)

    if ret:
        # 코너 좌표 더 정밀하게
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        objpoints.append(objp)
        imgpoints.append(corners2)

        print(f"OK: {fname}")
    else:
        print(f"FAIL: {fname} (체커보드 못 찾음)")

if len(objpoints) < 5:
    print("유효한 체커보드가 검출된 이미지가 너무 적어요 (최소 5장 이상 권장).")
    raise SystemExit

# 카메라 행렬(mtx), 왜곡 계수(dist) 추정
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

print("RMS reprojection error:", ret)
print("Camera Matrix (mtx):")
print(mtx)
print("Distortion Coeffs (dist):")
print(dist)

# 나중에 실시간 보정에 쓰려고 저장
np.savez("camera_calib_320x240.npz", mtx=mtx, dist=dist)
print("Saved calibration to camera_calib_320x240.npz")
